In [1]:
from model.data_utils import CoNLLDataset
from model.ner_model import NERModel
from model.config import Config

# create instance of config
config = Config()

# build model
model = NERModel(config)
model.build()
model.restore_session(config.dir_model)





D:\anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:96: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
Initializing tf session
Reloading the latest trained model...


INFO:tensorflow:Restoring parameters from results/test/model.weights/


Restoring parameters from results/test/model.weights/


In [2]:
import xml_reader
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from itertools import chain
x_data,lable = xml_reader.load_data_for_task_3('data/ABSA16_Restaurants_Train_SB1_v2.xml')

file_dir='data/sem_eval/test.txt'

with open(file_dir) as f:
    sentence=[]
    lable=[]
    temp=[]
    pos_tag=[]
    capital_tag=[]
    
    for line in f:
        temp.append(line)
        if line=='\n':
            temp_2=temp.copy()
            sentence_list=[]
            lable_list=[]
            pos_list=[]
            capital_list=[]
            for item in temp_2:
                item_splited=item.split(' ')
                if item_splited[0]=='\n':
                    continue
                sentence_list.append(item_splited[0])
                lable_list.append(item_splited[1])
                pos_list.append(item_splited[2])
                capital_item=item_splited[3].replace('\n','')
                capital_list.append(capital_item)
            sentence.append(sentence_list)
            lable.append(lable_list)
            pos_tag.append(pos_list)
            capital_tag.append(capital_list)
            temp=[]



test_set=sentence
test_lable=lable


print("DONE")




DONE


In [3]:
print(sentence[0])
print(pos_tag[0])
print(capital_tag[0])

['Love', 'Al', 'Di', 'La']
['NNP', 'NNP', 'NNP', 'NNP']
['10010000', '10001000', '10001000', '10001000']


In [3]:
import nltk
def bio_classification_report(y_true, y_pred):
    """
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.

    Note that it requires scikit-learn 0.15+ (or a version from github master)
    to calculate averages properly!
    """
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
        digits=4,
    )

predict_y=[]



for item_list,pos_list,capital_list in zip(test_set,pos_tag, capital_tag):
    result=model.predict(item_list,pos_list,capital_list)
    predict_y.append(result)
    
print(bio_classification_report(test_lable, predict_y))


f=open("data/sem_eval/test_result.txt",'w',encoding='UTF8')

for test_set_list,label, predicted_list in zip(test_set,test_lable,predict_y ):
    test_set_string=' '.join(test_set_list)
    label_string=' '.join(label)
    predicted_string=' '.join(predicted_list)
    
    f.write(test_set_string+'\n'+label_string+'\n'+predicted_string+'\n\n')
f.close()

case 1
[0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0]
case 1
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
             precision    recall  f1-score   support

   TARGET-B     0.7706    0.6222    0.6885       540
   TARGET-I     0.7382    0.5127    0.6052       275

avg / total     0.7597    0.5853    0.6604       815

